#  Projeto Deep Learning

## Importações

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import zipfile


from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Input



## Carregamento do CSV e Verificação de imagens

In [ ]:
gdrive=True
if gdrive:
    from google.colab import drive
    drive.mount('/content/drive')
    path = "/content/drive/MyDrive/DL/DL/"
else:
    path = "."

Mounted at /content/drive


In [ ]:
metadata_path = "/content/drive/MyDrive/DL/metadata.csv"
meta_data = pd.read_csv(metadata_path)

#base_dir = "/Users/joaosantos/Documents/Mestrado Joao/2 semestre/Deep Learning/rare_specie"

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DL/metadata.csv'

In [ ]:
if gdrive:
  # Extract the files into the "rare_species" folder
  zip_path = path + '/rare_species_div.zip'
  z = zipfile.ZipFile(zip_path)

  # Create the extraction folder if it does not exist
  os.makedirs("rare_species_div", exist_ok=True)

  # Extract the files into the "rare_species" folder
  z.extractall("rare_species_div")

  del z

In [ ]:
base_dir= "/content/rare_species_div"

In [ ]:
def check_image_exists(image_file):
    for root, dirs, files in os.walk(base_dir):
        if image_file in files:
            return True
    return False

In [ ]:
missing_images = []
for index, row in meta_data.iterrows():
    image_path = row['file_path']
    image_name = os.path.basename(image_path)
    if not check_image_exists(image_name):
        missing_images.append(image_path)

print(f"Número de imagens em falta: {len(missing_images)}")

In [ ]:
# Função para remover fundo da imagem
def remove_background(input_path, output_path):
    try:
        print(f"Removendo fundo da imagem: {input_path}")
        # Executa o rembg para remover o fundo
        result = subprocess.run(['rembg', 'i', input_path, output_path], check=True, capture_output=True, text=True)
        if result.returncode == 0:
            print(f"Fundo removido com sucesso de {input_path}. Salvo em {output_path}")
        else:
            print(f"Erro ao processar {input_path}: {result.stderr}")
    except subprocess.CalledProcessError as e:
        print(f"Erro ao processar {input_path}: {e}")

## Load Images
Carregar imagens visto que cada pasta é uma classe (familia) e dar one hot enconding nas classes , ou seja,
X_data são imagens e y_data são rótulos/classes codificados


Resizing usando 224x224, pois medidas mais baixas fazem perder qualidade de imagem e consequentemente pioram a performance

In [ ]:
# Definições
image_size = (224, 224)
batch_size = 32

# Augmentation para treino
train_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Apenas preprocessamento para val
val_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [ ]:
base_path=os.getcwd()+ '/rare_species_div'
base_path

'/content/rare_species_div'

In [ ]:
train_dir = base_path+ '/train'
val_dir = base_path+ '/val'

In [ ]:

train_dataset = train_aug.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_dataset = val_aug.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Verificar o shape dos batches
for X_batch, y_batch in train_dataset:
    print(f"X_train batch shape: {X_batch.shape}, y_train batch shape: {y_batch.shape}")
    break

for X_batch, y_batch in val_dataset:
    print(f"X_val batch shape: {X_batch.shape}, y_val batch shape: {y_batch.shape}")
    break

# Número de classes
num_classes = train_dataset.num_classes
print(f"Num classes: {num_classes}")

Found 8627 images belonging to 202 classes.
Found 2157 images belonging to 202 classes.
X_train batch shape: (32, 224, 224, 3), y_train batch shape: (32, 202)
X_val batch shape: (32, 224, 224, 3), y_val batch shape: (32, 202)
Num classes: 202


## Class Weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
labels = train_dataset.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights calculados:", class_weight_dict)

Class weights calculados: {0: np.float64(0.39544371103777043), 1: np.float64(1.9412691269126914), 2: np.float64(0.9932074602809118), 3: np.float64(2.0337105139085336), 4: np.float64(2.0337105139085336), 5: np.float64(0.20434411862238855), 6: np.float64(0.9932074602809118), 7: np.float64(1.9412691269126914), 8: np.float64(1.9412691269126914), 9: np.float64(1.9412691269126914), 10: np.float64(1.9412691269126914), 11: np.float64(1.9412691269126914), 12: np.float64(1.9412691269126914), 13: np.float64(1.9412691269126914), 14: np.float64(1.9412691269126914), 15: np.float64(1.9412691269126914), 16: np.float64(1.9412691269126914), 17: np.float64(0.5024461269656377), 18: np.float64(0.6570449352627571), 19: np.float64(0.9932074602809118), 20: np.float64(1.9412691269126914), 21: np.float64(2.0337105139085336), 22: np.float64(2.0337105139085336), 23: np.float64(0.9932074602809118), 24: np.float64(0.9932074602809118), 25: np.float64(0.33106915342697063), 26: np.float64(0.9932074602809118), 27: np.f

## Models

### EfficientNetB0

Sem Weigths:

In [ ]:
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# Callbacks
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("melhor_modelo.keras", save_best_only=True, monitor="val_loss", verbose=1)

#Treinamento, com base congelada
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=2
)

#Fine-tuning
base_model.trainable = True
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

#Treinamento, com base descongelada
fine_tune_history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stop, reduce_lr],
    verbose=2
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_2']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (115600000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Epoch 1: val_loss improved from inf to 4.40035, saving model to melhor_modelo.keras
300/300 - 476s - 2s/step - AUC: 0.6697 - accuracy: 0.0797 - loss: 4.9489 - val_AUC: 0.8178 - val_accuracy: 0.2165 - val_loss: 4.4003 - learning_rate: 1.0000e-04
Epoch 2/30

Epoch 2: val_loss improved from 4.40035 to 3.69721, saving model to melhor_modelo.keras
300/300 - 382s - 1s/step - AUC: 0.8025 - accuracy: 0.1902 - loss: 4.2049 - val_AUC: 0.9018 - val_accuracy: 0.3062 - val_loss: 3.6972 - learning_rate: 1.0000e-04
Epoch 3/30

Epoch 3: val_loss improved from 3.69721 to 3.14595, saving model to melhor_modelo.keras
300/300 - 374s - 1s/step - AUC: 0.8749 - accuracy: 0.2650 - loss: 3.6167 - val_AUC: 0.9427 - val_accuracy: 0.3663 - val_loss: 3.1460 - learning_rate: 1.0000e-04
Epoch 4/30

Epoch 4: val_loss improved from 3.14595 to 2.71600, saving model to melhor_modelo.keras
300/300 - 367s - 1s/step - AUC: 0.9186 - accuracy: 0.3300 - loss: 3.1371 - val_AUC: 0.9609 - val_accuracy: 0.4414 - val_loss: 2.7160

In [ ]:
Com Weights

In [ ]:
# === MODELO BASE ===
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# === CALLBACKS ===
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("melhor_modelo.keras", save_best_only=True, monitor="val_loss", verbose=1)

# === TREINO COM BASE CONGELADA ===
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=2,
    class_weight=class_weight_dict
)

# === FINE-TUNING ===
base_model.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

fine_tune_history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stop, reduce_lr],
    verbose=2,
    class_weight=class_weight_dict
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (115600000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Epoch 1: val_loss improved from inf to 4.92201, saving model to melhor_modelo.keras
270/270 - 444s - 2s/step - AUC: 0.5898 - accuracy: 0.0159 - loss: 5.2948 - val_AUC: 0.7569 - val_accuracy: 0.1349 - val_loss: 4.9220 - learning_rate: 1.0000e-04
Epoch 2/30

Epoch 2: val_loss improved from 4.92201 to 4.49208, saving model to melhor_modelo.keras
270/270 - 357s - 1s/step - AUC: 0.7429 - accuracy: 0.0802 - loss: 4.8475 - val_AUC: 0.8822 - val_accuracy: 0.3018 - val_loss: 4.4921 - learning_rate: 1.0000e-04
Epoch 3/30

Epoch 3: val_loss improved from 4.49208 to 3.92398, saving model to melhor_modelo.keras
270/270 - 352s - 1s/step - AUC: 0.8491 - accuracy: 0.1844 - loss: 4.3256 - val_AUC: 0.9398 - val_accuracy: 0.3959 - val_loss: 3.9240 - learning_rate: 1.0000e-04
Epoch 4/30

Epoch 4: val_loss improved from 3.92398 to 3.35126, saving model to melhor_modelo.keras
270/270 - 356s - 1s/step - AUC: 0.9010 - accuracy: 0.2738 - loss: 3.7413 - val_AUC: 0.9572 - val_accuracy: 0.4478 - val_loss: 3.3513

### ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# === MODELO BASE (ResNet50) ===
base_model_rn = ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model_rn.trainable = False  # Congelar as camadas convolucionais

# Adicionar camadas de classificação
x = base_model_rn.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout para evitar overfitting
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)

outputs = Dense(num_classes, activation="softmax")(x)  # Saída com softmax

# Criar o modelo final
model_rn = Model(inputs=base_model_rn.input, outputs=outputs)

# Compilar o modelo
model_rn.compile(
    optimizer=Adam(learning_rate=1e-4),  # Definir uma taxa de aprendizado inicial
    loss="categorical_crossentropy",  # Função de perda para múltiplas classes
    metrics=["accuracy", "AUC"]
)

# === CALLBACKS ===
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("melhor_modelo.keras", save_best_only=True, monitor="val_loss", verbose=1)

# === TREINAMENTO COM BASE CONGELADA ===
history = model_rn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=25,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=2,
    class_weight=class_weight_dict  # Considerando os pesos das classes
)

# === FINE-TUNING ===
base_model_rn.trainable = True  # Descongelar as camadas convolucionais para fine-tuning

# Recompilar o modelo com uma taxa de aprendizado menor para o fine-tuning
model_rn.compile(
    optimizer=Adam(learning_rate=1e-5),  # Reduzir a taxa de aprendizado para fine-tuning
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# Fine-tuning - treinamento com as camadas superiores descongeladas
fine_tune_history = model_rn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stop, reduce_lr],
    verbose=2,
    class_weight=class_weight_dict  # Considerando os pesos das classes
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (115600000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Epoch 1: val_loss improved from inf to 5.09961, saving model to melhor_modelo.keras
270/270 - 402s - 1s/step - AUC: 0.5544 - accuracy: 0.0131 - loss: 5.6980 - val_AUC: 0.6552 - val_accuracy: 0.0487 - val_loss: 5.0996 - learning_rate: 1.0000e-04
Epoch 2/25

Epoch 2: val_loss improved from 5.09961 to 4.87766, saving model to melhor_modelo.keras
270/270 - 381s - 1s/step - AUC: 0.6319 - accuracy: 0.0275 - loss: 5.1583 - val_AUC: 0.7397 - val_accuracy: 0.1080 - val_loss: 4.8777 - learning_rate: 1.0000e-04
Epoch 3/25

Epoch 3: val_loss improved from 4.87766 to 4.45475, saving model to melhor_modelo.keras
270/270 - 331s - 1s/step - AUC: 0.6917 - accuracy: 0.0562 - loss: 4.8984 - val_AUC: 0.8290 - val_accuracy: 0.1734 - val_loss: 4.4548 - learning_rate: 1.0000e-04
Epoch 4/25

Epoch 4: val_loss improved from 4.45475 to 3.89858, saving model to melhor_modelo.keras
270/270 - 333s - 1s/step - AUC: 0.7575 - accuracy: 0.0982 - loss: 4.5311 - val_AUC: 0.8790 - val_accuracy: 0.2272 - val_loss: 3.8986

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# === MODELO BASE (ResNet50) ===
base_model_rn = ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model_rn.trainable = False  # Congelar as camadas convolucionais

# Adicionar camadas de classificação
x = base_model_rn.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout para evitar overfitting
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)  # Saída com softmax

# Criar o modelo final
model_rn = Model(inputs=base_model_rn.input, outputs=outputs)

# Compilar o modelo
model_rn.compile(
    optimizer=Adam(learning_rate=5e-4),  # Definir uma taxa de aprendizado inicial
    loss="categorical_crossentropy",  # Função de perda para múltiplas classes
    metrics=["accuracy", "AUC"]
)

# === CALLBACKS ===
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("melhor_modelo.keras", save_best_only=True, monitor="val_loss", verbose=1)


# === FINE-TUNING ===
base_model_rn.trainable = True  # Descongelar as camadas convolucionais para fine-tuning

# Recompilar o modelo com uma taxa de aprendizado menor para o fine-tuning
model_rn.compile(
    optimizer=Adam(learning_rate=5e-5),  # Reduzir a taxa de aprendizado para fine-tuning
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# Fine-tuning - treinamento com as camadas superiores descongeladas
fine_tune_history = model_rn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stop, reduce_lr],
    verbose=2,
    class_weight=class_weight_dict  # Considerando os pesos das classes
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (115600000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


270/270 - 511s - 2s/step - AUC: 0.5670 - accuracy: 0.0195 - loss: 5.4866 - val_AUC: 0.7330 - val_accuracy: 0.0872 - val_loss: 4.7522 - learning_rate: 5.0000e-05
Epoch 2/10
270/270 - 388s - 1s/step - AUC: 0.7082 - accuracy: 0.0811 - loss: 4.7205 - val_AUC: 0.8612 - val_accuracy: 0.2137 - val_loss: 3.8909 - learning_rate: 5.0000e-05
Epoch 3/10
270/270 - 379s - 1s/step - AUC: 0.8151 - accuracy: 0.1713 - loss: 3.8756 - val_AUC: 0.9097 - val_accuracy: 0.3162 - val_loss: 3.1920 - learning_rate: 5.0000e-05
Epoch 4/10
270/270 - 381s - 1s/step - AUC: 0.8773 - accuracy: 0.2612 - loss: 3.1840 - val_AUC: 0.9324 - val_accuracy: 0.3839 - val_loss: 2.6796 - learning_rate: 5.0000e-05
Epoch 5/10
270/270 - 372s - 1s/step - AUC: 0.9056 - accuracy: 0.3382 - loss: 2.6536 - val_AUC: 0.9431 - val_accuracy: 0.4182 - val_loss: 2.5070 - learning_rate: 5.0000e-05
Epoch 6/10
270/270 - 375s - 1s/step - AUC: 0.9301 - accuracy: 0.4022 - loss: 2.2306 - val_AUC: 0.9432 - val_accuracy: 0.4622 - val_loss: 2.2844 - learn

### DenseNet121

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Base model: DenseNet121
base_model = DenseNet121(include_top=False, weights="imagenet", input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False  # Fase 1: feature extractor

# Classifier head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)

# Compilar o modelo (fase 1)
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# Callbacks
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("melhor_modelo_densenet.keras", save_best_only=True, monitor="val_loss", verbose=1)

# Treinamento com base congelada
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=2
)

# Fine-tuning: descongelar a base
base_model.trainable = True

# Compilar novamente com learning rate menor
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# Treinamento com base descongelada
fine_tune_history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stop, reduce_lr],
    verbose=2
)


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (115600000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Epoch 1: val_loss improved from inf to 5.31984, saving model to melhor_modelo_densenet.keras
270/270 - 448s - 2s/step - AUC: 0.5247 - accuracy: 0.0115 - loss: 11.5109 - val_AUC: 0.5950 - val_accuracy: 0.0343 - val_loss: 5.3198 - learning_rate: 1.0000e-04
Epoch 2/30

Epoch 2: val_loss improved from 5.31984 to 5.26354, saving model to melhor_modelo_densenet.keras
270/270 - 380s - 1s/step - AUC: 0.5489 - accuracy: 0.0165 - loss: 6.8928 - val_AUC: 0.5648 - val_accuracy: 0.0301 - val_loss: 5.2635 - learning_rate: 1.0000e-04
Epoch 3/30

Epoch 3: val_loss did not improve from 5.26354
270/270 - 363s - 1s/step - AUC: 0.5654 - accuracy: 0.0170 - loss: 5.8307 - val_AUC: 0.5923 - val_accuracy: 0.0343 - val_loss: 5.2769 - learning_rate: 1.0000e-04
Epoch 4/30

Epoch 4: val_loss did not improve from 5.26354
270/270 - 362s - 1s/step - AUC: 0.5677 - accuracy: 0.0206 - loss: 5.5234 - val_AUC: 0.6163 - val_accuracy: 0.0362 - val_loss: 5.2765 - learning_rate: 1.0000e-04
Epoch 5/30

Epoch 5: ReduceLROnPla

KeyboardInterrupt: 

Check some information about scores:

In [ ]:
# from tensorflow.keras.models import load_model
# from sklearn.metrics import classification_report, confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt
# import numpy as np

# # Carregar o modelo salvo
# modelo_final = load_model("melhor_modelo.keras")

# # Previsões
# y_pred_probs = modelo_final.predict(X_val)
# y_pred = np.argmax(y_pred_probs, axis=1)
# y_true = np.argmax(y_val, axis=1)

# # Classification report
# print("Classification Report:")
# print(classification_report(y_true, y_pred))

# # Matriz de confusão
# plt.figure(figsize=(8,6))
# sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap="Blues")
# plt.title("Matriz de Confusão")
# plt.xlabel("Previsto")
# plt.ylabel("Real")
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd

# --- 1. Guardar histórico do treino ---

def save_history(history, fine_tune_history, filename='training_history.csv'):
    # Junta os históricos
    def extract_data(hist):
        return pd.DataFrame(hist.history)

    combined_history = pd.concat([extract_data(history), extract_data(fine_tune_history)], ignore_index=True)
    combined_history.to_csv(filename, index=False)
    print(f"Histórico guardado em {filename}")

# --- 2. Plotar curvas de treino ---

def plot_history(history, fine_tune_history):
    def merge(hist1, hist2, metric):
        return hist1.history[metric] + hist2.history[metric]

    epochs_range = range(len(history.history['loss']) + len(fine_tune_history.history['loss']))

    plt.figure(figsize=(18, 5))

    # Loss
    plt.subplot(1, 3, 1)
    plt.plot(epochs_range, merge(history, fine_tune_history, 'loss'), label='Treino')
    plt.plot(epochs_range, merge(history, fine_tune_history, 'val_loss'), label='Validação')
    plt.title('Loss')
    plt.legend()

    # Accuracy
    plt.subplot(1, 3, 2)
    plt.plot(epochs_range, merge(history, fine_tune_history, 'accuracy'), label='Treino')
    plt.plot(epochs_range, merge(history, fine_tune_history, 'val_accuracy'), label='Validação')
    plt.title('Accuracy')
    plt.legend()

    # AUC
    plt.subplot(1, 3, 3)
    plt.plot(epochs_range, merge(history, fine_tune_history, 'AUC'), label='Treino')
    plt.plot(epochs_range, merge(history, fine_tune_history, 'val_AUC'), label='Validação')
    plt.title('AUC')
    plt.legend()

    plt.tight_layout()
    plt.show()

# --- 3. Matriz de confusão ---

def plot_confusion_matrix(model, dataset, class_names):
    # Gerar previsões
    y_true = []
    y_pred = []

    for images, labels in dataset:
        preds = model.predict(images)
        y_true.extend(np.argmax(labels, axis=1))
        y_pred.extend(np.argmax(preds, axis=1))

        if len(y_true) >= dataset.samples:
            break

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=False, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Matriz de Confusão')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    print("\nRelatório de Classificação:")
    print(classification_report(y_true, y_pred, target_names=class_names))

# --- 4. Guardar modelo final ---

def save_model(model, filename='modelo_final.keras'):
    model.save(filename)
    print(f"Modelo final guardado em {filename}")

# =========================================================
# 🚀 Usa as funções depois de treinares:

# Guardar histórico
save_history(history, fine_tune_history)

# Plotar as curvas
plot_history(history, fine_tune_history)

# Matriz de confusão
plot_confusion_matrix(model, val_dataset, list(train_dataset.class_indices.keys()))

# Guardar o modelo final
save_model(model)


------------------------------

### Com gaussian blur

In [ ]:
base_path=os.getcwd()+ '/rare_species_div'
base_path

'/content/rare_species_div'

In [ ]:
train_dir = base_path+ '/train'
val_dir = base_path+ '/val'

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input

def custom_preprocessing(image):
    # Converte para uint8 se necessário
    image = image.astype(np.uint8)

    # Aplica Gaussian Blur (kernel 3x3, pode ajustar para 5x5)
    image = cv2.GaussianBlur(image, (3, 3), 0)

    # Converte de volta para float32
    image = image.astype(np.float32)

    # Aplica preprocess_input (ex. para EfficientNet)
    return preprocess_input(image)


In [ ]:
train_aug = ImageDataGenerator(
    preprocessing_function=custom_preprocessing,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_aug = ImageDataGenerator(
    preprocessing_function=custom_preprocessing
)


In [ ]:
# Definições
image_size = (224, 224)
batch_size = 32

In [ ]:
train_dataset = train_aug.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_dataset = val_aug.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Verificar o shape dos batches
for X_batch, y_batch in train_dataset:
    print(f"X_train batch shape: {X_batch.shape}, y_train batch shape: {y_batch.shape}")
    break

for X_batch, y_batch in val_dataset:
    print(f"X_val batch shape: {X_batch.shape}, y_val batch shape: {y_batch.shape}")
    break

# Número de classes
num_classes = train_dataset.num_classes
print(f"Num classes: {num_classes}")

Found 8627 images belonging to 202 classes.
Found 2157 images belonging to 202 classes.
X_train batch shape: (32, 224, 224, 3), y_train batch shape: (32, 202)
X_val batch shape: (32, 224, 224, 3), y_val batch shape: (32, 202)
Num classes: 202


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
labels = train_dataset.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights calculados:", class_weight_dict)

Class weights calculados: {0: np.float64(0.39544371103777043), 1: np.float64(1.9412691269126914), 2: np.float64(0.9932074602809118), 3: np.float64(2.0337105139085336), 4: np.float64(2.0337105139085336), 5: np.float64(0.20434411862238855), 6: np.float64(0.9932074602809118), 7: np.float64(1.9412691269126914), 8: np.float64(1.9412691269126914), 9: np.float64(1.9412691269126914), 10: np.float64(1.9412691269126914), 11: np.float64(1.9412691269126914), 12: np.float64(1.9412691269126914), 13: np.float64(1.9412691269126914), 14: np.float64(1.9412691269126914), 15: np.float64(1.9412691269126914), 16: np.float64(1.9412691269126914), 17: np.float64(0.5024461269656377), 18: np.float64(0.6570449352627571), 19: np.float64(0.9932074602809118), 20: np.float64(1.9412691269126914), 21: np.float64(2.0337105139085336), 22: np.float64(2.0337105139085336), 23: np.float64(0.9932074602809118), 24: np.float64(0.9932074602809118), 25: np.float64(0.33106915342697063), 26: np.float64(0.9932074602809118), 27: np.f

In [ ]:
# === MODELO BASE ===
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# === CALLBACKS ===
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("melhor_modelo.keras", save_best_only=True, monitor="val_loss", verbose=1)

# === TREINO COM BASE CONGELADA ===
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=2,
    class_weight=class_weight_dict
)

# === FINE-TUNING ===
base_model.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

fine_tune_history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stop, reduce_lr],
    verbose=2,
    class_weight=class_weight_dict
)

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (115600000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Epoch 1: val_loss improved from inf to 4.98355, saving model to melhor_modelo.keras
270/270 - 408s - 2s/step - AUC: 0.5863 - accuracy: 0.0158 - loss: 5.2980 - val_AUC: 0.7306 - val_accuracy: 0.0890 - val_loss: 4.9835 - learning_rate: 1.0000e-04
Epoch 2/30

Epoch 2: val_loss improved from 4.98355 to 4.61814, saving model to melhor_modelo.keras
270/270 - 340s - 1s/step - AUC: 0.7228 - accuracy: 0.0626 - loss: 4.9040 - val_AUC: 0.8515 - val_accuracy: 0.2378 - val_loss: 4.6181 - learning_rate: 1.0000e-04
Epoch 3/30

Epoch 3: val_loss improved from 4.61814 to 4.11829, saving model to melhor_modelo.keras
270/270 - 329s - 1s/step - AUC: 0.8179 - accuracy: 0.1476 - loss: 4.4737 - val_AUC: 0.9143 - val_accuracy: 0.3338 - val_loss: 4.1183 - learning_rate: 1.0000e-04
Epoch 4/30

Epoch 4: val_loss improved from 4.11829 to 3.59071, saving model to melhor_modelo.keras
270/270 - 382s - 1s/step - AUC: 0.8770 - accuracy: 0.2215 - loss: 3.9582 - val_AUC: 0.9365 - val_accuracy: 0.3802 - val_loss: 3.5907